## Analysis_Caffeine
This project demonstrated the usage of pkdb for the test substance caffeine.
### Data Sources
- pkdb rest api : "http://0.0.0.0:8000/api/v1/"

### Outputs:
- data/processed/outputs.tsv
- data/processed/timecourses.tsv
- data/processed/interventions.tsv
- data/processed/individuals.tsv
- data/processed/groups.tsv

#### Groups and Individuals 
- data/processed/all_subjects.tsv

### Merged Dataframes
- data/processed/all_subjects.tsv

### Normalized Dataframes
- data/processed/all_subjects.tsv

### Changes
- 11-29-2018 : Started project

In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
from utils import PkdbModel , Preprocessed
import pandas as pd


In [43]:
queries = ['outputs','timecourses','interventions','individuals','groups',"studies","substances"]
#queries = ["studies","substances"]

### Query, Process and Save
Json data is loaded, transformed to dataframes, preprocessed and saved as tab seperated files.

In [44]:
for pk_instance in queries:
    pk_instance = PkdbModel(pk_instance)  
    pk_instance.load()
    pk_instance.preprocess()
    pk_instance.save()
    pk_instance.report()

____________________________________________________________
Name: outputs
Loaded: True
Preprocessed: True
saved: True
outputs were succsesfully saved to </home/janekg89/Develop/Pycharm_Projects/pkdb_analysis/data/1-preprocessed/outputs.tsv>
____________________________________________________________
Name: timecourses
Loaded: True
Preprocessed: True
saved: True
timecourses were succsesfully saved to </home/janekg89/Develop/Pycharm_Projects/pkdb_analysis/data/1-preprocessed/timecourses.tsv>
____________________________________________________________
Name: interventions
Loaded: True
Preprocessed: True
saved: True
interventions were succsesfully saved to </home/janekg89/Develop/Pycharm_Projects/pkdb_analysis/data/1-preprocessed/interventions.tsv>
____________________________________________________________
Name: individuals
Loaded: True
Preprocessed: True
saved: True
individuals were succsesfully saved to </home/janekg89/Develop/Pycharm_Projects/pkdb_analysis/data/1-preprocessed/individ

### Convert Units


In [ ]:
this_data = PkdbModel('timecourses',destination='1-preprocessed')
this_data.read()
this_data.data.groupby(["pktype","unit"]).count()

### Merge
 - merge individuals and groups  --> all_subjects,
 - merge outputs and timecourses --> all_results, 
 - merge all_results with interventions and individuals  --> individuals_complete, 
 - merge all_results with interventions and groups  --> groups_complete, 
 - merge all_results with interventions and all_subjects -->  all_complete

In [6]:
prepocessed = Preprocessed()

In [7]:
prepocessed.read()
prepocessed.merge()
prepocessed.save()

/home/janekg89/Develop/Pycharm_Projects/pkdb_analysis/utils.py:381: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat([self.individuals.data,self.groups.data], keys=["individual","group"])
/home/janekg89/Develop/Pycharm_Projects/pkdb_analysis/utils.py:392: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat([self.outputs.data,self.timecourses.data], keys=["outputs","timecourses"])
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended